In [1]:
import random

class Team:
  def __init__(self, name, wins, games, points, win_rate):
    self.name = name
    self.points = points
    self.wins = wins
    self.games = games
    self.win_rate = win_rate

  def __repr__(self):
    return (self.name +" : " + str(self.points) + " / "+ str(self.wins)+ " / "+ str(self.games)+ " / "+ str(self.win_rate) + "\n")

def search_team_one(gameString,teamList,team_one):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team.name == teams[0]):      
            return team

def search_team_two(gameString,teamList,team_two):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team.name == teams[1]):      
            return team

In [2]:
def simulate_game(team_one,team_two,current_team_list):
  status = "UNDEFINED"
  rng = random.uniform(0,1)
  rng_two = random.uniform(0,1)
  team_one.games = team_one.games + 1
  team_two.games = team_two.games + 1
  if(team_one.win_rate >= rng):
    if(team_two.win_rate < rng_two):
      team_one.points = team_one.points + 3  
      team_one.wins = team_one.wins + 1
      team_one.win_rate = team_one.wins / team_one.games
      status = "ONE WON"   
    else:
      team_one.points = team_one.points + 1
      team_two.points = team_two.points + 1
      status = "TIE"
  else:
    if(team_two.win_rate >= rng_two):
      team_two.points = team_two.points + 3
      team_two.wins = team_two.wins + 1
      team_two.win_rate = team_two.wins / team_two.games
      status = "TWO WON"
    else:
      team_one.points = team_one.points + 1
      team_two.points = team_two.points + 1
      status = "TIE"
  for group in current_team_list:
      for team in group:
        if(team.name == team_one.name):
          team = team_one
        if(team.name == team_two.name):
          team = team_two
      group = group 

def simulate_game_elm(team_one,team_two,current_team_list):
  status = "UNDEFINED"
  team_one.games = team_one.games + 1
  team_two.games = team_two.games + 1

  while(status == "UNDEFINED" or status == "TIE"):
        rng = random.uniform(0,1)
        rng_two = random.uniform(0,1)  
        if(team_one.win_rate >= rng):
            if(team_two.win_rate < rng_two):      
              team_one.wins = team_one.wins + 1
              team_one.win_rate = team_one.wins / team_one.games
              current_team_list.pop(current_team_list.index(team_two)) 
              status = "WIN"
            else:     
              status = "TIE"
        else:
            if(team_two.win_rate >= rng_two):      
              team_two.wins = team_two.wins + 1
              team_two.win_rate = team_two.wins / team_two.games
              current_team_list.pop(current_team_list.index(team_one)) 
              status = "WIN"
            else:      
              status = "TIE" 
    
  return current_team_list 

In [141]:
%%time
brazil_group = []
second_phase = []
second_phase_teams = []
k_simulations = 10000
counter = 0 
            
for i in range(0,k_simulations): 
  winner_list = []
  base_name = "group"
  original_team_list = [[],[],[],[],[],[],[],[]]
  for j in range (0,8):
    current_file = base_name + str(j)+ ".txt"
    f = open(current_file, "r",encoding="utf-8-sig")        
    for line in f.readlines():
        ele = line.split(";")
        if(ele[0] != '\n'):
            ele[3] = ele[3][:-1]         
            temp_team = Team(ele[0],int(ele[2]),int(ele[3]),int(ele[1]),int(ele[2])/int(ele[3]),)
            original_team_list[j].append(temp_team)
            
  current_team_list = original_team_list.copy()
  t = open("jogos_copa.txt", "r",encoding="utf-8-sig")
  team_one = Team("",0,0,0,0)
  team_two = Team("",0,0,0,0)  
        
  for game in t.readlines():  
    team_one = search_team_one(game,current_team_list,team_one)
    team_two = search_team_two(game,current_team_list,team_two)  
    simulate_game(team_one,team_two,current_team_list)
 
  for group in current_team_list:
    group.sort(key=lambda x: x.points, reverse=True)  
    
  for group in current_team_list:
    winner_list.append(group[0])
    winner_list.append(group[1])

  if(i == 0):
    for group in current_team_list:     
        second_phase_teams.append(group[0])
        second_phase_teams.append(group[1])  
    
  if(current_team_list[6][0].name == "Brasil" or current_team_list[6][1].name == "Brasil"):
        counter = counter + 1
        
  found_win_list =  False
  current_win_list = ""
  for team in winner_list:
    current_win_list = current_win_list + team.name + ";"
  for groupAndCounter in second_phase:     
    if(groupAndCounter[0] == current_win_list):
      found_win_list = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found_win_list):
    second_phase.append([current_win_list,1])
        
  found = False
  current_group_string = current_team_list[6][0].name+";"+current_team_list[6][1].name+";"+current_team_list[6][2].name+";"+current_team_list[6][3].name+";"
  for groupAndCounter in brazil_group:     
    if(groupAndCounter[0] == current_group_string):
      found = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found):
    brazil_group.append([current_group_string,1])      
        
for group in original_team_list:
    print(group)

[Catar : 5 / 83 / 205 / 0.4088669950738916, Holanda : 5 / 192 / 460 / 0.4192139737991266, Equador : 4 / 91 / 258 / 0.35271317829457366, Senegal : 1 / 75 / 186 / 0.4098360655737705]
[Estados Unidos : 7 / 161 / 426 / 0.3779342723004695, Inglaterra : 5 / 234 / 429 / 0.5480093676814989, Gales : 2 / 50 / 166 / 0.3067484662576687, Irã : 1 / 92 / 198 / 0.4717948717948718]
[Arábia Saudita : 5 / 98 / 243 / 0.4049586776859504, México : 5 / 248 / 521 / 0.47784200385356457, Argentina : 3 / 293 / 587 / 0.5017123287671232, Polônia : 1 / 224 / 542 / 0.4155844155844156]
[França : 9 / 226 / 508 / 0.4448818897637795, Dinamarca : 6 / 252 / 542 / 0.46494464944649444, Austrália : 1 / 144 / 314 / 0.4630225080385852, Tunísia : 1 / 102 / 279 / 0.3695652173913043]
[Espanha : 7 / 185 / 346 / 0.5362318840579711, Japão : 5 / 105 / 253 / 0.4166666666666667, Alemanha : 4 / 310 / 589 / 0.5263157894736842, Costa Rica : 0 / 77 / 247 / 0.3155737704918033]
[Croácia : 7 / 73 / 151 / 0.48344370860927155, Marrocos : 5 / 10

In [147]:
brazil_group.sort(key=lambda x: x[1], reverse=True)   
for group in brazil_group:
    group[1] = group[1] / k_simulations
    print(group)    

print(counter/k_simulations)                

['Brasil;Sérvia;Suíça;Camarões;', 2.268e-05]
['Brasil;Suíça;Sérvia;Camarões;', 1.0860000000000001e-05]
['Brasil;Sérvia;Camarões;Suíça;', 1.083e-05]
['Brasil;Camarões;Sérvia;Suíça;', 8.71e-06]
['Brasil;Suíça;Camarões;Sérvia;', 7.6e-06]
['Sérvia;Brasil;Suíça;Camarões;', 6.2e-06]
['Suíça;Brasil;Sérvia;Camarões;', 4.08e-06]
['Brasil;Camarões;Suíça;Sérvia;', 4.07e-06]
['Sérvia;Brasil;Camarões;Suíça;', 3.97e-06]
['Camarões;Brasil;Sérvia;Suíça;', 2.76e-06]
['Sérvia;Suíça;Brasil;Camarões;', 2.75e-06]
['Suíça;Brasil;Camarões;Sérvia;', 2.41e-06]
['Sérvia;Camarões;Brasil;Suíça;', 2.38e-06]
['Camarões;Brasil;Suíça;Sérvia;', 1.76e-06]
['Sérvia;Suíça;Camarões;Brasil;', 1.6400000000000002e-06]
['Suíça;Camarões;Brasil;Sérvia;', 1.37e-06]
['Suíça;Sérvia;Brasil;Camarões;', 1.16e-06]
['Suíça;Sérvia;Camarões;Brasil;', 1.14e-06]
['Camarões;Sérvia;Brasil;Suíça;', 8.8e-07]
['Camarões;Sérvia;Suíça;Brasil;', 8.3e-07]
['Sérvia;Camarões;Suíça;Brasil;', 7.100000000000001e-07]
['Camarões;Suíça;Brasil;Sérvia;', 5.6

In [114]:
second_phase.sort(key=lambda x: x[1], reverse=True)   
for group in second_phase:
    group[1] = group[1] / k_simulations
    print(group)    
    

['Catar;Senegal;Irã;Inglaterra;Argentina;Polônia;Dinamarca;Tunísia;Alemanha;Japão;Marrocos;Croácia;Brasil;Camarões;Portugal;Uruguai;', 0.01]
['Catar;Holanda;Irã;Estados Unidos;Polônia;Argentina;Austrália;Dinamarca;Espanha;Alemanha;Canadá;Croácia;Brasil;Camarões;Uruguai;Gana;', 0.01]
['Catar;Senegal;Inglaterra;Irã;Argentina;Arábia Saudita;França;Dinamarca;Espanha;Costa Rica;Croácia;Bélgica;Brasil;Sérvia;Uruguai;Portugal;', 0.01]
['Catar;Equador;Estados Unidos;Gales;Argentina;México;França;Dinamarca;Costa Rica;Japão;Marrocos;Bélgica;Brasil;Camarões;Coreia do Sul;Gana;', 0.01]
['Catar;Holanda;Inglaterra;Gales;Argentina;Polônia;Tunísia;França;Japão;Costa Rica;Marrocos;Bélgica;Brasil;Suíça;Portugal;Uruguai;', 0.01]
['Catar;Equador;Inglaterra;Irã;México;Argentina;Dinamarca;Tunísia;Espanha;Alemanha;Bélgica;Canadá;Suíça;Camarões;Portugal;Coreia do Sul;', 0.01]
['Equador;Catar;Inglaterra;Gales;México;Polônia;Austrália;França;Japão;Espanha;Croácia;Marrocos;Brasil;Suíça;Gana;Coreia do Sul;', 0.01

In [154]:
print(second_phase_teams)
coup_winners = []
for i in range(0,k_simulations):    
    second_phase_teams_current = second_phase_teams.copy()
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    second_team = second_phase_teams_current[2]
    third_team = second_phase_teams_current[3]
    fourth_team = second_phase_teams_current[4]
    fifth_team = second_phase_teams_current[5]
    sixth_team = second_phase_teams_current[6]
    seventh_team = second_phase_teams_current[7]
    
    eight_team = second_phase_teams_current[8]
    nineth_team = second_phase_teams_current[9]
    tenth_team = second_phase_teams_current[10]
    eleventh_team = second_phase_teams_current[11]
    twelveth_team = second_phase_teams_current[12]
    thirteenth_team = second_phase_teams_current[13]
    fourteenth_team = second_phase_teams_current[14]
    fifteenth_team = second_phase_teams_current[15]
    

    second_phase_teams_current = simulate_game_elm(zeroth_team,third_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(first_team,second_team,second_phase_teams_current) 
    second_phase_teams_current = simulate_game_elm(fourth_team,seventh_team,second_phase_teams_current)   
    second_phase_teams_current = simulate_game_elm(fifth_team,sixth_team,second_phase_teams_current)    
    second_phase_teams_current = simulate_game_elm(eight_team,eleventh_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(nineth_team,tenth_team,second_phase_teams_current) 
    second_phase_teams_current = simulate_game_elm(twelveth_team,fifteenth_team,second_phase_teams_current)   
    second_phase_teams_current = simulate_game_elm(thirteenth_team,fourteenth_team,second_phase_teams_current) 
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    second_team = second_phase_teams_current[2]
    third_team = second_phase_teams_current[3]
    fourth_team = second_phase_teams_current[4]
    fifth_team = second_phase_teams_current[5]
    sixth_team = second_phase_teams_current[6]
    seventh_team = second_phase_teams_current[7]
    
    second_phase_teams_current = simulate_game_elm(zeroth_team,third_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(first_team,second_team,second_phase_teams_current) 
    second_phase_teams_current = simulate_game_elm(fourth_team,seventh_team,second_phase_teams_current)   
    second_phase_teams_current = simulate_game_elm(fifth_team,sixth_team,second_phase_teams_current)
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    second_team = second_phase_teams_current[2]
    third_team = second_phase_teams_current[3]
    
    second_phase_teams_current = simulate_game_elm(zeroth_team,third_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(first_team,second_team,second_phase_teams_current) 
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    
    second_phase_teams_current = simulate_game_elm(zeroth_team,first_team,second_phase_teams_current) 
    
    found_winner = False
    for teamAndCounter in coup_winners:     
        if(teamAndCounter[0] == second_phase_teams_current[0]):
          found_winner = True
          teamAndCounter[1] = teamAndCounter[1] + 1
    if(not found_winner):
        coup_winners.append([second_phase_teams_current[0],1])  

coup_winners.sort(key=lambda x: x[1], reverse=True)   
for group in coup_winners:
    group[1] = group[1] / k_simulations
    print(group)    
    


[Holanda : 7 / 3399 / 13641 / 0.24930321255684318
, Equador : 5 / 966 / 11126 / 0.0869956772334294
, Inglaterra : 5 / 22232 / 30521 / 0.728440366972477
, Irã : 5 / 15394 / 24771 / 0.6214775938635446
, Argentina : 4 / 9874 / 19931 / 0.4954588790205229
, Arábia Saudita : 4 / 5398 / 15486 / 0.3486854854337575
, Tunísia : 7 / 9379 / 19366 / 0.48432739478440484
, Dinamarca : 3 / 5461 / 15688 / 0.3481226493274686
, Alemanha : 7 / 25354 / 32799 / 0.7730349411549484
, Espanha : 4 / 9617 / 19537 / 0.4922706797706798
, Croácia : 5 / 3979 / 14011 / 0.28407224958949095
, Canadá : 4 / 518 / 10671 / 0.048601989116156874
, Brasil : 5 / 25887 / 32588 / 0.7943721615318522
, Sérvia : 5 / 11018 / 20752 / 0.5309879518072289
, Portugal : 7 / 3167 / 13327 / 0.23765571064085247
, Coreia do Sul : 4 / 755 / 10956 / 0.06902541598098373
]
[Brasil : 5 / 52768 / 66158 / 0.7976057317331238
, 0.3311]
[Alemanha : 7 / 52068 / 66371 / 0.7845110742805484
, 0.3142]
[Inglaterra : 5 / 45830 / 62079 / 0.7382647636843971
, 0